# jit, vmap, grad, and pytrees

All `LinearOperator` objects are native jax and pytorch pytrees.

This means that we can vmap over them, jit functions with them, as well as other operations.

## Example: Tree Map



## Jit example (in jax)

Let's choose jitting a function involving matrix square roots.

In [1]:
from jax import jit
import numpy as np
import jax.numpy as jnp
import jax.config as config
# set cpu
config.update("jax_platform_name", "cpu")
import cola

# construct a linear operator
A = jnp.array(np.random.randn(2, 2))
B = cola.SelfAdjoint(cola.lazify(A.T@A+ 1e-4*jnp.eye(2)))
D = cola.SelfAdjoint(cola.diag(jnp.array([3.,0.2, 1.])))
K = cola.kron(B,D)

Lets verify that CoLA indeed computes the sqrts on this matrix

In [2]:
v = jnp.array(np.random.randn(6))
K_half_v = cola.sqrt(K, tol=1e-4)@v
Kv = cola.sqrt(K, tol=1e-4)@K_half_v
print("error:",jnp.linalg.norm(Kv - K@v))

/home/marc_f/anaconda3/envs/cola/lib/python3.10/site-packages/beartype/_util/hint/pep/utilpeptest.py:311: BeartypeDecorHintPep585DeprecationWarning: PEP 484 type hint typing.Callable deprecated by PEP 585. This hint is scheduled for removal in the first Python version released after October 5th, 2025. To resolve this, import this hint from "beartype.typing" rather than "typing". For further commentary and alternatives, see also:
    https://beartype.readthedocs.io/en/latest/api_roar/#pep-585-deprecations
  warn(


error: 3.7252903e-09


Now let's jit a function with a `LinearOperator` as an argument

In [3]:
@jit
def sqrt_mvm(K, v):
    return cola.sqrt(K, tol=1e-4)@v

print(sqrt_mvm(K,v))
print(sqrt_mvm(4*K,v)/2)

[ 0.01154765 -0.10519305  0.08695581  0.12265506  0.49678737 -0.61266834]
[ 0.01154772 -0.10519368  0.08695581  0.12265505  0.49678725 -0.61266834]


## Batched LinearOperator operations using vmap (in pytorch)

Let's consider a function that constructs some linear operators, and a separate function that applies some transformations.


In [10]:
import cola
import torch
import numpy as np

def construct_complicated_linops(X):
    X = cola.lazify(X)
    Y = X@X.T
    Y = cola.PSD(Y+cola.ops.I_like(Y))
    D = cola.PSD(cola.diag(torch.linspace(0.1,1,2)))
    W = cola.ops.BlockDiag(Y,D, multiplicities=[2, 1])
    diag_W = cola.diag(W)
    return W, cola.PSD(cola.diag(diag_W))

W,diag_W = construct_complicated_linops(torch.randn(3,3))
print(W[:5,:5].to_dense())
print(diag_W[:5,:5].to_dense())

tensor([[1.8414, 0.6225, 0.8826, 0.0000, 0.0000],
        [0.6225, 2.4913, 1.5865, 0.0000, 0.0000],
        [0.8826, 1.5865, 3.7751, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 1.8414, 0.6225],
        [0.0000, 0.0000, 0.0000, 0.6225, 2.4913]])
tensor([[1.8414, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 2.4913, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 3.7751, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 1.8414, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 2.4913]])


For the example, let's consider a function that uses the diagonal of `W` as a symmetric preconditioner, used explicitly (rather than supplied as an argument to inverse).

In [11]:
def perform_operations(W,D,v):
    P = cola.pow(D, -0.5) # D^{-1/2}
    y = P@cola.inverse(P@W@P,tol=1e-4)@P@v
    return y 

Now suppose that we want to perform this operation over a batch of LinearOperators, each with different data.

First we can vmap over the functions constructing the LinearOperators:

In [24]:
from torch.func import vmap
bW, bD = vmap(construct_complicated_linops)(torch.randn(3,5, 5))

Notice that the objects are the same types and shapes,

In [25]:
print(bW.shape, type(bW))
print(bD.shape, type(bD))

(12, 12) <class 'cola.ops.operators.BlockDiag[cola.ops.operators.Sum[cola.ops.operators.Product[cola.ops.operators.Dense, cola.ops.operators.Transpose[cola.ops.operators.Dense]], cola.ops.operators.Identity], cola.ops.operators.Diagonal]'>
(12, 12) <class 'cola.ops.operators.Diagonal'>


However the data the makes up these objects now has a batch dimension:

In [26]:
bD.diag.shape

torch.Size([3, 12])

In general these objects should not be used except precisely in conjunction with a function which is vmapped over a LinearOperator input, as shown below with perform_operations.

In [27]:
all_outs = torch.func.vmap(perform_operations)(bW, bD, torch.randn(3,bW.shape[0]))
print(all_outs.shape)

torch.Size([3, 12])


🚧 Note: Not all LinearOperators with pytorch backend support vmap 🚧

For example kronecker:

In [29]:
def get_entries(M):
    return M[:5,:5].to_dense()

try:
    vmap(get_entries)(vmap(cola.kron)(bW, bD))
except RuntimeError as e:
    print("raised exception:", e)

raised exception: Batching rule not implemented for aten::moveaxis.int; the fallback path doesn't work on out= or view ops.


## Gradients and PyTrees (jax example)